In [4]:
import requests
from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize
import copy
import os
from zipfile import ZipFile
import pandas as pd
import yaml
import time
import linecache
import sys
import json
import math

ModuleNotFoundError: No module named 'pandas'

In [2]:
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print ('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

In [3]:
def yaml_dump(filepath, data): #Dumps file
    with open(filepath, "w") as file_descriptor:
        yaml.dump(data, file_descriptor)
def yaml_loader(filepath): #Load a file
    with open(filepath, "r") as file_descriptor:
        data = yaml.load(file_descriptor,Loader=yaml.FullLoader)
    return data


In [4]:
class MyError(Exception): 
    def __init__(self, value): 
        self.value = value 
    def __str__(self):
        return(repr(self.value)) 

In [5]:
def modifyTour(Name):
    x0 = pd.read_csv(Name)
    if 'tour' in x0['match_type'][0]:
        a = set(x0['period'])
        if len(a)<=2:
            b = set(x0['over.number'])
            if max(b) > 20:
                x0['match_type']='One-Day Internationals'
                print(max(b),"ODI NOW")
            else:
                x0['match_type']='Twenty20 Internationals'
                print(max(b),"T20 NOW")
        else:
            x0['match_type']='Tests'
            print(len(a),"TEST NOW")
        x0.to_csv("{} match data/{}".format(season,i))
        print("SAVED")
    else:
        print("ALL CLEAR")
    print("DONE")

In [6]:
def DataFetch():
    payload = {'key':'b432b9e7b0339c476ddba32a29e4644c', 
               'url': 'https://httpbin.org/ip'}
    matchData = []
    periodnum = 0
    if(len(season)>4):
        Fyr = season[:4]
        Syr = season[-2:]
        seasonVal=Fyr+"%2F"+Syr
    else:
        seasonVal = season
    pageUrl = "http://www.espncricinfo.com/ci/engine/series/index.html?season="+seasonVal+";view=season"
    page = requests.get(pageUrl, params = payload).text
    soup = BeautifulSoup(page, 'html.parser')
    print('Starting for season : ' + season)
    #print("page html: ", soup.prettify())
    matchSectionDiv = soup.find_all('div', class_='match-section-head')
    match_type = [ 'Tests', 'One-Day Internationals', 'Twenty20 Internationals', 'First-class', 'List A', 'Twenty20', 
                   "Women's Tests", "Women's One-Day Internationals", "Women's Twenty20 Internationals",'Other matches', 
                   'Other one-day/limited-overs matches', 'Other Twenty20 matches', 'tour']
    # 'Tests', 'One-Day Internationals', 'Twenty20 Internationals', 'First-class', 'List A', 'Twenty20', 
    #               "Women's Tests", "Women's One-Day Internationals", "Women's Twenty20 Internationals",'Other matches', 
    #               'Other one-day/limited-overs matches', 'Other Twenty20 matches', 'tour'
    for matchSection in matchSectionDiv:
        if matchSection.get_text() not in match_type:
            continue
        if matchSection.get_text() in ['Tests','First-class', 'Other matches', "Women's Tests"]:
            periodnum = 4
        else:
            periodnum = 2
        seriesSummarySection = matchSection.findNext('section', class_="series-summary-wrap")
        allSeriesSections = seriesSummarySection.find_all('section', class_="series-summary-block")
        for seriesSection in allSeriesSections:
            count=0
            seriesFullUrl = ""
            seriesUrl = seriesSection['data-summary-url']
            seriesFullUrl = "http://www.espncricinfo.com" + seriesUrl

            # seriesFullUrl = http://www.espncricinfo.com/ci/engine/match/index/series.html?series=10233
            seriesPage = requests.get(seriesFullUrl, params = payload).text
            seriesSoup = BeautifulSoup(seriesPage, 'html.parser')
            dateInfo = seriesSoup.find_all('div',class_ = "match-info")
            seriesData = seriesSoup.find_all('span',class_ = "match-no")
            for match,dateVal in zip(seriesData,dateInfo):             
                matchURL = match.find('a',href=True)
                date = dateVal.findNext('span',class_ = 'bold').get_text()
                matchVal = matchURL['href'].split('/')
                Type = matchSection.get_text()
                if 'Women' in Type:
                    matchData.append([date,matchURL.get_text(),matchVal[4],matchVal[6],'Women',Type,periodnum])
                else:
                    matchData.append([date,matchURL.get_text(),matchVal[4],matchVal[6],Type,periodnum])


    for i in matchData:
        index = i[1].find(' at ')
        temp = list(i[1])
        name = temp[6:index+3]
        del temp[:index+4]
        del temp[-6:]
        i[1] = ''.join(temp)
        name = ''.join(name)+i[1]
        i.insert(4,name)
        if 'Only' in i[4]:
            i[4] = i[4].replace("Only","1st")
        nameFile = list(i[4])
        if '/' in nameFile:
            while '/' in nameFile:
                nameFile.remove('/')
        i[4] = ''.join(nameFile)
        for j in range(2,4):
            if(i[j][-1]=='/'):
                temp = list(i[j])
                temp.remove('/')
                i[j] = ''.join(temp)

    if(len(matchData)==0):
        raise(MyError("Data not extracted"))
    print(len(matchData))
    yaml_dump("Match List/{}_match_list.yaml".format(season),matchData)
    temp = yaml_loader("Match List/{}_match_list.yaml".format(season))
    print("Matches stored in file for season {} is {}".format(season,len(temp)))
    temp.clear()

In [7]:
def DataCollection():
    global matchData
    matchData = yaml_loader("Match List/{}_match_list.yaml".format(season))
    if os.path.isdir("{} match data".format(season)) == False:
        print("Creating folder for match data")
        os.mkdir("{} match data".format(season))
    count = 0
    total = len(matchData)
    for eventNo in matchData:
        if len(eventNo) == 8:
            FileName = "Women's {} season- {}.csv".format(eventNo[0][-4:],eventNo[4])
        else:
            FileName = "{} season- {}.csv".format(eventNo[0][-4:],eventNo[4])
        print("Match: ",count+1,"of Total: ",total)
        print("Date: ",eventNo[0],"\nVenue: ",eventNo[1])
        print("Match Type: ",eventNo[-2],"\nInnings: ",eventNo[-1])
        if(os.path.isfile("{} match data/{}".format(season,FileName)))==True:
            if 'tour' in eventNo[-2]:
                print("Modifying Tour")
                modifyTour("{} match data/{}".format(season,FileName))
            count+=1
            continue
        seriesNo = eventNo[2]
        matchNo = eventNo[3]
        url = 'https://site.web.api.espn.com/apis/site/v2/sports/cricket/'+seriesNo+'/playbyplay'

        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
        payload = {
        'contentorigin': 'espn',
        'event': matchNo,
        'page': '1',
        'period': '1',
        'section': 'cricinfo',
        'key':'b432b9e7b0339c476ddba32a29e4644c', 
        'url':'https://httpbin.org/ip'}
        # Get inital page of data, including total number of pages to iterate through
        response = requests.get(url, headers=headers, params=payload).json()
        pageCount = response['commentary']['pageCount']



        #Store the initial page to jsonData, iterate through the next `pageCount` pages and add that to the list for a final result
        jsonData = response
        for i in range(1,eventNo[-1]+1):
            print("innings: ", i)
            prd = str(i)
            payload = {
            'contentorigin': 'espn',
            'event': matchNo,
            'page': '1',
            'period': prd,
            'section': 'cricinfo',
            'key':'b432b9e7b0339c476ddba32a29e4644c', 
            'url':'https://httpbin.org/ip'}
            response = requests.get(url, headers=headers, params=payload).json()
            pageCount = response['commentary']['pageCount']
            print ('Total pages: %s\n' %(pageCount))
            for page in range(1,pageCount+1):
                payload = {
                        'contentorigin': 'espn',
                        'event': matchNo,
                        'page': page,
                        'period': prd,
                        'section': 'cricinfo',
                        'api_key':'b432b9e7b0339c476ddba32a29e4644c',
                        'url':'https://httpbin.org/ip'}

                response = requests.get(url, headers=headers, params=payload).json()
                if 'commentary' not in response.keys():
                    continue
                jsonData['commentary']['items'] += response['commentary']['items']
                print ('Processed page: %s' %page)
        

        df = json_normalize(jsonData['commentary']['items'])
        
        df['matchId'] = matchNo
        df['seriesId'] = seriesNo
        df['match_type'] = eventNo[-2]
        
        tempName = eventNo[4]
        df.to_csv("{} match data/{}".format(season,FileName))
        if len(df.axes[0])<=2:
            os.remove("{} match data/{}".format(season,FileName))
            print("Deleting {} for name {}".format(FileName,tempName))
            matchData = [i for i in matchData if tempName not in i]
        count+=1
    print(len(matchData))
    if 'tour' in eventNo[-2]:
        modifyTour("{} match data/{}".format(season,FileName))
    yaml_dump("Match List/{}_match_list.yaml".format(season),matchData)
    %store  matchData
    %store season
    Done = []
    print("Entering second program")
    %run matchCleaning.ipynb
    print("Entering third program")
    %run matchData.ipynb

In [35]:
season = input("Enter season: ")
for i in range(1, 20):
    try:
        matchData=[]
        if os.path.isfile("Match List/{}_match_list.yaml".format(season)):
            print("Match List for {} season already exists".format(season))
            DataCollection()
        else:
            DataFetch()
            DataCollection()
    except MyError as e:
        PrintException()
        if os.path.isfile("Match List/{}_match_list.yaml".format(season)):
            temp = yaml_loader("Match List/{}_match_list.yaml".format(season))
        if len(temp) == 0:
            os.remove("Match List/{}_match_list.yaml".format(season))
        print('Restarting!')
        continue
    except:
        PrintException()
        yaml_dump("Match List/{}_match_list.yaml".format(season),matchData)
        print ('Restarting!')
        continue
    else:
        break

Enter season: 2019
Match List for 2019 season already exists
Match:  1 of Total:  945
Date:  Jul 24-26, 2019 
Venue:  Lord's, London
Match Type:  Tests 
Innings:  4
Match:  2 of Total:  945
Date:  Aug 1-5, 2019 
Venue:  Edgbaston, Birmingham
Match Type:  Tests 
Innings:  4
Match:  3 of Total:  945
Date:  Aug 1-5, 2019 
Venue:  Edgbaston, Birmingham
Match Type:  Tests 
Innings:  4
Match:  4 of Total:  945
Date:  Apr 10, 2019 
Venue:  Harare Sports Club
Match Type:  One-Day Internationals 
Innings:  2
Match:  5 of Total:  945
Date:  Apr 12, 2019 
Venue:  Harare Sports Club
Match Type:  One-Day Internationals 
Innings:  2
Match:  6 of Total:  945
Date:  Apr 14, 2019 
Venue:  Harare Sports Club
Match Type:  One-Day Internationals 
Innings:  2
Match:  7 of Total:  945
Date:  Apr 16, 2019 
Venue:  Harare Sports Club
Match Type:  One-Day Internationals 
Innings:  2
Match:  8 of Total:  945
Date:  May 3, 2019 
Venue:  The Village, Malahide, Dublin
Match Type:  One-Day Internationals 
Innings: 

Match:  170 of Total:  945
Date:  May 14-17, 2019 
Venue:  Edgbaston, Birmingham
Match Type:  First-class 
Innings:  4
Match:  171 of Total:  945
Date:  May 20-22, 2019 
Venue:  County Ground, Taunton
Match Type:  First-class 
Innings:  4
Match:  172 of Total:  945
Date:  May 20-23, 2019 
Venue:  New Close County Cricket Ground, Newport, Isle-of-Wight
Match Type:  First-class 
Innings:  4
Match:  173 of Total:  945
Date:  May 20-23, 2019 
Venue:  Kent County Cricket Ground, Beckenham
Match Type:  First-class 
Innings:  4
Match:  174 of Total:  945
Date:  May 27-30, 2019 
Venue:  County Ground, Chelmsford
Match Type:  First-class 
Innings:  4
Match:  175 of Total:  945
Date:  May 27-30, 2019 
Venue:  Edgbaston, Birmingham
Match Type:  First-class 
Innings:  4
Match:  176 of Total:  945
Date:  May 27-30, 2019 
Venue:  Headingley, Leeds
Match Type:  First-class 
Innings:  4
Match:  177 of Total:  945
Date:  Jun 3-6, 2019 
Venue:  Woodbridge Road, Guildford
Match Type:  First-class 
Inning

Match Type:  First-class 
Innings:  4
Match:  254 of Total:  945
Date:  Jun 30-Jul 3, 2019 
Venue:  County Ground, Derby
Match Type:  First-class 
Innings:  4
Match:  255 of Total:  945
Date:  Jun 30-Jul 3, 2019 
Venue:  Sophia Gardens, Cardiff
Match Type:  First-class 
Innings:  4
Match:  256 of Total:  945
Date:  Jul 7-10, 2019 
Venue:  Grace Road, Leicester
Match Type:  First-class 
Innings:  4
Match:  257 of Total:  945
Date:  Jul 7-9, 2019 
Venue:  Merchant Taylors' School Ground, Northwood
Match Type:  First-class 
Innings:  4
Match:  258 of Total:  945
Date:  Jul 7-10, 2019 
Venue:  County Ground, Northampton
Match Type:  First-class 
Innings:  4
Match:  259 of Total:  945
Date:  Jul 7-10, 2019 
Venue:  Chester Road North Ground, Kidderminster
Match Type:  First-class 
Innings:  4
Match:  260 of Total:  945
Date:  Jul 13-16, 2019 
Venue:  Sophia Gardens, Cardiff
Match Type:  First-class 
Innings:  4
Match:  261 of Total:  945
Date:  Jul 13-15, 2019 
Venue:  Old Trafford, Manches

Match Type:  List A 
Innings:  2
Match:  379 of Total:  945
Date:  Apr 26, 2019 
Venue:  Wanderers Cricket Ground, Windhoek
Match Type:  List A 
Innings:  2
Match:  380 of Total:  945
Date:  Apr 27, 2019 
Venue:  United Cricket Club Ground, Windhoek
Match Type:  List A 
Innings:  2
Match:  381 of Total:  945
Date:  Apr 27, 2019 
Venue:  Wanderers Cricket Ground, Windhoek
Match Type:  List A 
Innings:  2
Match:  382 of Total:  945
Date:  Apr 27, 2019 
Venue:  Affies Park, Windhoek
Match Type:  List A 
Innings:  2
Match:  383 of Total:  945
Date:  Apr 24, 2019 
Venue:  La Manga Club Top Ground
Match Type:  List A 
Innings:  2
Match:  384 of Total:  945
Date:  Apr 25, 2019 
Venue:  La Manga Club Top Ground
Match Type:  List A 
Innings:  2
Match:  385 of Total:  945
Date:  May 23, 2019 
Venue:  Bready Cricket Club, Magheramason, Bready
Match Type:  List A 
Innings:  2
Match:  386 of Total:  945
Date:  May 27, 2019 
Venue:  Civil Service Cricket Club, Stormont, Belfast
Match Type:  List A 


Match Type:  Twenty20 
Innings:  2
Match:  504 of Total:  945
Date:  Aug 1, 2019 
Venue:  Lord's, London (night)
Match Type:  Twenty20 
Innings:  2
Match:  505 of Total:  945
Date:  Aug 1, 2019 
Venue:  County Ground, Chelmsford (night)
Match Type:  Twenty20 
Innings:  2
Match:  506 of Total:  945
Date:  Aug 1, 2019 
Venue:  Sophia Gardens, Cardiff (night)
Match Type:  Twenty20 
Innings:  2
Match:  507 of Total:  945
Date:  Aug 2, 2019 
Venue:  Trent Bridge, Nottingham (night)
Match Type:  Twenty20 
Innings:  2
Match:  508 of Total:  945
Date:  Aug 2, 2019 
Venue:  The Cooper Associates County Ground, Taunton (night)
Match Type:  Twenty20 
Innings:  2
Match:  509 of Total:  945
Date:  Aug 2, 2019 
Venue:  County Ground, Bristol (night)
Match Type:  Twenty20 
Innings:  2
Match:  510 of Total:  945
Date:  Aug 2, 2019 
Venue:  Riverside Ground, Chester-le-Street (night)
Match Type:  Twenty20 
Innings:  2
Match:  511 of Total:  945
Date:  Aug 2, 2019 
Venue:  Headingley, Leeds (night)
Matc

...

559
2019 match cleaned/2019 season- North Group atEdgbaston, Birmingham (night).yaml
Twenty20 Warwickshire Derbyshire
560
2019 match cleaned/2019 season- North Group atCounty Ground, Derby (night).yaml
Twenty20 Nottinghamshire Derbyshire
561
2019 match cleaned/2019 season- South Group atThe Cooper Associates County Ground, Taunton (night).yaml
Twenty20 Somerset Hampshire
562
2019 match cleaned/2019 season- North Group atEdgbaston, Birmingham.yaml
List A Warwickshire Yorkshire
563
2019 match cleaned/2019 season- North Group atGrace Road, Leicester.yaml
List A Leicestershire Worcestershire
564
2019 match cleaned/2019 season- South Group atThe Rose Bowl, Southampton.yaml
List A Glamorgan Hampshire
565
2019 match cleaned/2019 season- North Group atCounty Ground, New Road, Worcester.yaml
List A Durham Worcestershire
566
2019 match cleaned/2019 season- North Group atTrent Bridge, Nottingham.yaml
List A Nottinghamshire Lancashire
567
2019 match cleaned/2019 season- South Group atSophia Garden

620
2019 match cleaned/2019 season- Group B atTribhuvan University International Cricket Ground, Kirtipur.yaml
Other one-day/limited-overs matches Tribhuwan Army Club Province Number 2
621
2019 match cleaned/2019 season- Group B atMulpani Cricket Ground.yaml
Other one-day/limited-overs matches Nepal Police Club Karnali Province
622
2019 match cleaned/2019 season- Group A atTribhuvan University International Cricket Ground, Kirtipur.yaml
Other one-day/limited-overs matches Armed Police Force Club Province Number 3
623
2019 match cleaned/2019 season- Group B atTribhuvan University International Cricket Ground, Kirtipur.yaml
Other one-day/limited-overs matches Tribhuwan Army Club Province Number 2
624
2019 match cleaned/2019 season- Group A atMulpani Cricket Ground.yaml
Other one-day/limited-overs matches Province Number 5 Province Number 3
625
2019 match cleaned/2019 season- Group B atTribhuvan University International Cricket Ground, Kirtipur.yaml
Other one-day/limited-overs matches Tri

669
2019 match cleaned/2019 season- 6th Match atHazelaarweg, Rotterdam.yaml
Other one-day/limited-overs matches Ireland Under-19s Denmark Under-19s
670
2019 match cleaned/2019 season- 8th Match atSportpark Westvliet, The Hague.yaml
Other one-day/limited-overs matches Jersey Under-19s Denmark Under-19s
671
2019 match cleaned/2019 season- 7th Match atSportpark Thurlede, Schiedam.yaml
Other one-day/limited-overs matches Ireland Under-19s Netherlands Under-19s
672
2019 match cleaned/2019 season- 10th Match atSportpark Thurlede, Schiedam.yaml
Other one-day/limited-overs matches Netherlands Under-19s Denmark Under-19s
673
2019 match cleaned/2019 season- 11th Match atSportpark Westvliet, The Hague.yaml
Other one-day/limited-overs matches Scotland Under-19s Ireland Under-19s
674
2019 match cleaned/2019 season- 14th Match atSportpark Westvliet, The Hague.yaml
Other one-day/limited-overs matches Scotland Under-19s Denmark Under-19s
675
2019 match cleaned/2019 season- 13th Match atSportpark Thurl

723
2019 match cleaned/2019 season- Tour Match atOld Hararians, Harare.yaml
One-Day Internationals Zimbabwe Chairman's XI United Arab Emirates
724
2019 match cleaned/2019 season- 1st ODI atHarare Sports Club.yaml
One-Day Internationals United Arab Emirates Zimbabwe
725
2019 match cleaned/2019 season- 2nd ODI atHarare Sports Club.yaml
One-Day Internationals United Arab Emirates Zimbabwe
726
2019 match cleaned/2019 season- 3rd ODI atHarare Sports Club.yaml
One-Day Internationals Zimbabwe United Arab Emirates
727
2019 match cleaned/2019 season- 4th ODI atHarare Sports Club.yaml
One-Day Internationals United Arab Emirates Zimbabwe
728
2019 match cleaned/2019 season- 1st ODI atCivil Service Cricket Club, Stormont, Belfast.yaml
One-Day Internationals Ireland Afghanistan
729
2019 match cleaned/2019 season- 2nd ODI atCivil Service Cricket Club, Stormont, Belfast.yaml
One-Day Internationals Afghanistan Ireland
730
2019 match cleaned/2019 season- 1st T20I atSportpark Maarschalkerweerd, Utrecht.y

780
2019 match cleaned/2019 season- 2nd unofficial ODI atGroenkloof Oval.yaml
Other one-day/limited-overs matches South Africa Emerging Players Women Bangladesh Emerging Players Women
781
2019 match cleaned/2019 season- 3rd unofficial ODI atGroenkloof Oval.yaml
Other one-day/limited-overs matches South Africa Emerging Players Women Bangladesh Emerging Players Women
782
2019 match cleaned/2019 season- 1st unofficial T20I atHammanskraal Mandela Oval.yaml
Other Twenty20 matches South Africa Emerging Players Women Bangladesh Emerging Players Women
783
2019 match cleaned/2019 season- 2nd unofficial T20I atHammanskraal Mandela Oval.yaml
Other Twenty20 matches Bangladesh Emerging Players Women South Africa Emerging Players Women
784
2019 match cleaned/2019 season- 3rd unofficial T20I atHammanskraal Mandela Oval.yaml
Other Twenty20 matches South Africa Emerging Players Women Bangladesh Emerging Players Women
785
2019 match cleaned/2019 season- Tour Match atP Sara Oval, Colombo.yaml
One-Day Int